**Before running this notebook, please add a shortcut of the RecSys folder to the root directory of your drive (MyDrive)**

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


The following is for the KAN layer.

In [2]:
!git clone https://github.com/ZiyaoLi/fast-kan

!pip install '/content/fast-kan/.'

Cloning into 'fast-kan'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (32/32), done.
remote: Total 202 (delta 52), reused 62 (delta 44), pack-reused 124 (from 1)
Receiving objects: 100% (202/202), 420.57 KiB | 1.83 MiB/s, done.
Resolving deltas: 100% (91/91), done.
Processing ./fast-kan
  Preparing metadata (setup.py) ... done
  Created wheel for fastkan: filename=fastkan-0.0.1-py3-none-any.whl size=11560 sha256=dff3d9df3946ad947baccc370c7f645c07fcfd479841c4912ebd49221bc6448e
  Stored in directory: /root/.cache/pip/wheels/90/67/af/0e9541499b9ccc4465dae8c80e40769cc637277b82bcfeb08b
Successfully built fastkan


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/RecSys/')
sys.path.append('/content/drive/MyDrive/RecSys/model/')

In [4]:
import math
import random as rnd
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
from torch.autograd import Variable
from torch.utils.data import DataLoader, Subset,random_split,default_collate
from sklearn.model_selection import train_test_split
import time
import logging


In [5]:
import matplotlib.pyplot as plt
import seaborn as sns
from numba import jit

In [6]:
from AutoRec_KAN import AE_KAN
from MovieLens_4Colab import Movie_1M, CombinedDataset, preprocess_100K



In [7]:
def custom_collate(batch, batch_size, dataset):
    if len(batch) < batch_size:

        additional_samples_needed = batch_size - len(batch)
        indices = np.random.choice(len(dataset), additional_samples_needed)
        additional_data = [dataset[idx] for idx in indices]
        batch.extend(additional_data)
    return default_collate(batch)

In [8]:
torch.manual_seed(4224)

np.random.seed(4224)

The following is for construction 9 different 9:1 train-test splits.

In [9]:
#dataset = Movie_1M()

In [10]:
#len(dataset)

In [11]:
# cross_validation

#for sd in [1,2,3,5,8,13,21,34]:
#
#  indices = np.random.permutation(len(dataset))
#  train_index = indices[:int(round(0.9*len(dataset)))]
#  train_rating, train_mask, test_rating, test_mask = dataset.preprocessor(train_index, fourier = False)

#  torch.save(train_mask, '/content/drive/MyDrive/RecSys/1m_train_mask_'+str(sd)+'.pt')
#  torch.save(train_rating, '/content/drive/MyDrive/RecSys/1m_train_rating_'+str(sd)+'.pt')

#  torch.save(test_rating, '/content/drive/MyDrive/RecSys/1m_test_rating_'+str(sd)+'.pt')
#  torch.save(test_mask, '/content/drive/MyDrive/RecSys/1m_test_mask_'+str(sd)+'.pt')



In [12]:
#from sklearn.model_selection import KFold


In [13]:
#kf = KFold(n_splits=10, shuffle=True, random_state=42)  # Set a random seed for reproducibility

# Enumerate over each split
'''
for fold, (train_index, test_index) in enumerate(kf.split(dataset)):
    train_rating, train_mask, test_rating, test_mask = dataset.preprocessor(train_index, fourier=False)

    torch.save(train_mask, f'/content/drive/MyDrive/RecSys/1m_train_mask_fold_{fold}.pt')
    torch.save(train_rating, f'/content/drive/MyDrive/RecSys/1m_train_rating_fold_{fold}.pt')

    torch.save(test_rating, f'/content/drive/MyDrive/RecSys/1m_test_rating_fold_{fold}.pt')
    torch.save(test_mask, f'/content/drive/MyDrive/RecSys/1m_test_mask_fold_{fold}.pt')
'''

"\nfor fold, (train_index, test_index) in enumerate(kf.split(dataset)):\n    train_rating, train_mask, test_rating, test_mask = dataset.preprocessor(train_index, fourier=False)\n\n    torch.save(train_mask, f'/content/drive/MyDrive/RecSys/1m_train_mask_fold_{fold}.pt')\n    torch.save(train_rating, f'/content/drive/MyDrive/RecSys/1m_train_rating_fold_{fold}.pt')\n\n    torch.save(test_rating, f'/content/drive/MyDrive/RecSys/1m_test_rating_fold_{fold}.pt')\n    torch.save(test_mask, f'/content/drive/MyDrive/RecSys/1m_test_mask_fold_{fold}.pt')\n"

In [14]:
model = AE_KAN(configuration_file = '/content/drive/MyDrive/RecSys/MovieLens1M.yaml')


 	 ----------- Model Loaded ------------
	 *Total Params* =  36439504
	 *Trainable Params* =  36439488


The following blocks are all for data pre-processing. Change the random seed to change the train/test sets for validation.

In [15]:
fold = 9

In [16]:
train_mask = torch.load(f'/content/drive/MyDrive/RecSys/1m_train_mask_fold_{fold}.pt')
train_rating = torch.load(f'/content/drive/MyDrive/RecSys/1m_train_rating_fold_{fold}.pt')

test_rating = torch.load(f'/content/drive/MyDrive/RecSys/1m_test_rating_fold_{fold}.pt')
test_mask = torch.load(f'/content/drive/MyDrive/RecSys/1m_test_mask_fold_{fold}.pt')

<ipython-input-16-078528d1d7e5>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  train_mask = torch.load(f'/content/drive/MyDrive/RecSys/1m_train_mask_fold_{fold}.pt')
<ipyt

In [17]:
train_rating = torch.tensor(train_rating)
train_mask = torch.tensor(train_mask)
test_rating = torch.tensor(test_rating)
test_mask = torch.tensor(test_mask)


**Re-run the following codes for Cross-Validation after running new dataset construction codes at the bottom.**

In [18]:
batch_size = model.batch_size

In [19]:
combineset = CombinedDataset(train_rating, train_mask)
combine_testset = CombinedDataset(test_rating, test_mask)
dataset = DataLoader(combineset, batch_size = batch_size, shuffle = True,
                     collate_fn=lambda x: custom_collate(x, batch_size, combineset))
test_dataset = DataLoader(combine_testset, batch_size = batch_size, shuffle = False,
                     collate_fn=lambda x: custom_collate(x, batch_size, combineset))

### Remember to set a unique run_id to make fine-tuning easer

In [20]:
model.train(dataset, test_dataset, epoch =30)


Epoch 1: 100%|██████████| 12/12 [00:02<00:00,  5.50batch/s, loss=0.21]



	 Epoch : 1, Average Training RMSE = 2.0473, "Accuracy" = 0.2271
	 Training time for current epoch: 2.19 seconds
	 RMSE on testing set : 1.5866, RMSE (rounded): 1.6157 , MAE: 1.2655 , Accuracy: 0.2487


Epoch 2: 100%|██████████| 12/12 [00:01<00:00, 10.96batch/s, loss=0.136]



	 Epoch : 2, Average Training RMSE = 1.4073, "Accuracy" = 0.2978
	 Training time for current epoch: 1.1 seconds
	 RMSE on testing set : 1.2782, RMSE (rounded): 1.3113 , MAE: 1.0181 , Accuracy: 0.3046


Epoch 3: 100%|██████████| 12/12 [00:01<00:00, 11.68batch/s, loss=0.113]


...


	 Epoch : 28, Average Training RMSE = 0.7997, "Accuracy" = 0.4782
	 Training time for current epoch: 1.07 seconds
	 RMSE on testing set : 0.7901, RMSE (rounded): 0.8409 , MAE: 0.6243 , Accuracy: 0.4807


Epoch 29: 100%|██████████| 12/12 [00:01<00:00, 11.49batch/s, loss=0.0536]



	 Epoch : 29, Average Training RMSE = 0.8001, "Accuracy" = 0.4789
	 Training time for current epoch: 1.05 seconds
	 RMSE on testing set : 0.802, RMSE (rounded): 0.8524 , MAE: 0.6336 , Accuracy: 0.4765


Epoch 30: 100%|██████████| 12/12 [00:01<00:00, 10.42batch/s, loss=0.0546]



	 Epoch : 30, Average Training RMSE = 0.8006, "Accuracy" = 0.478
	 Training time for current epoch: 1.16 seconds
	 RMSE on testing set : 0.8091, RMSE (rounded): 0.8592 , MAE: 0.6404 , Accuracy: 0.4682


In [21]:
import gc
gc.collect()

0